# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
# Importing stock list
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2588496318287,
 'week52high': 169.4,
 'week52low': 119.9,
 'week52highSplitAdjustOnly': 172,
 'week52lowSplitAdjustOnly': 118.92,
 'week52change': 0.3669443064648631,
 'sharesOutstanding': 16502259768,
 'float': 0,
 'avg10Volume': 92762770,
 'avg30Volume': 76917336,
 'day200MovingAvg': 150.1,
 'day50MovingAvg': 154.26,
 'employees': 152853,
 'ttmEPS': 11.44,
 'ttmDividendRate': 0.8762997517101159,
 'dividendYield': 0.005563865435013969,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-29',
 'nextEarningsDate': '2022-01-21',
 'peRatio': 14.13838268259807,
 'beta': 1.4692198388307858,
 'maxChangePercent': 59.91475100455038,
 'year5ChangePercent': 5.080544235905614,
 'year2ChangePercent': 1.44019353407238,
 'year1ChangePercent': 0.3706517288974013,
 'ytdChangePercent': 0.1916008962588575,
 'month6ChangePercent': 0.2675715302500085,
 'month3ChangePercent': 0.0582062545982718,
 'month1ChangePercent': 0.03057354009064221,
 'day30ChangePercent': 

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.3706517288974013

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Batch API Call
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
         final_dataframe = final_dataframe.append(
             pd.Series(
                 [
                     symbol, 
                     data[symbol]['price'],
                     data[symbol]['stats']['year1ChangePercent'],
                     'N/A'
                 ], 
                 index = my_columns
             ), 
             ignore_index = True
         )
final_dataframe
    
    
    
    
    
    
    
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.87,0.344616,N/A
1,AAL,18.32,0.187601,N/A
2,AAP,233.44,0.547262,N/A
3,AAPL,160.88,0.359804,N/A
4,ABBV,120.90,0.168086,N/A
...,...,...,...,...
500,YUM,127.37,0.18348,N/A
501,ZBH,128.00,-0.165128,N/A
502,ZBRA,627.50,0.571535,N/A
503,ZION,65.40,0.686075,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
# Removing low-momentum stocks
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.65,2.332291,N/A
1,DVN,44.06,2.017648,N/A
2,FTNT,353.50,1.651769,N/A
3,MRO,16.13,1.56269,N/A
4,FANG,112.13,1.521952,N/A
5,MCHP,85.10,1.504918,N/A
6,NVDA,335.20,1.391486,N/A
7,F,20.38,1.188911,N/A
8,NUE,113.61,1.162783,N/A
9,IT,335.00,1.125495,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
# Calculating the number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Must enter a number. \nPlease enter a number.')
        portfolio_size = input('Enter the size of your portfolio: ')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000
10000000


In [25]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe
    

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.65,2.332291,237
1,DVN,44.06,2.017648,445
2,FTNT,353.50,1.651769,55
3,MRO,16.13,1.56269,1215
4,FANG,112.13,1.521952,174
5,MCHP,85.10,1.504918,230
6,NVDA,335.20,1.391486,58
7,F,20.38,1.188911,962
8,NUE,113.61,1.162783,172
9,IT,335.00,1.125495,58


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [26]:
 hqm_columns = [
     'Ticker',
     'Price',
     'Number of Shares to Buy',
     'One-Year Price Return',
     'One-Year Return Percentile',
     'Six-Month Price Return',
     'Six-Month Return Percentile',
     'Three-Month Price Return',
     'Three-Month Return Percentile',
     'One-Month Price Return',
     'One-Month Return Percentile',
     'HQM Score'
 ]
    
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
                index = hqm_columns
            ),
            ignore_index = True
        
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.04,N/A,0.354359,N/A,0.105264,N/A,-0.125512,N/A,-0.024066,N/A,N/A
1,AAL,18.16,N/A,0.18682,N/A,-0.272982,N/A,-0.128236,N/A,-0.075634,N/A,N/A
2,AAP,229.87,N/A,0.533743,N/A,0.207903,N/A,0.122241,N/A,0.007955,N/A,N/A
3,AAPL,167.46,N/A,0.370301,N/A,0.263445,N/A,0.057965,N/A,0.029574,N/A,N/A
4,ABBV,122.53,N/A,0.169847,N/A,0.055357,N/A,-0.01406,N/A,0.063361,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.54,N/A,0.180956,N/A,0.043751,N/A,-0.055557,N/A,-0.009572,N/A,N/A
501,ZBH,124.10,N/A,-0.159926,N/A,-0.252058,N/A,-0.1644,N/A,-0.138238,N/A,N/A
502,ZBRA,608.50,N/A,0.582308,N/A,0.189485,N/A,0.004724,N/A,0.111074,N/A,N/A
503,ZION,67.12,N/A,0.698432,N/A,0.128443,N/A,0.104987,N/A,0.027154,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [27]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe = hqm_dataframe.fillna(value = np.nan)
        hqm_dataframe.loc[row, percentile_col] = (score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]))/100
        #hqm_dataframe.dropna(inplace=True)
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.04,N/A,0.354359,0.633663,0.105264,0.675248,-0.125512,0.126733,-0.024066,0.39802,N/A
1,AAL,18.16,N/A,0.186820,0.427723,-0.272982,0.035644,-0.128236,0.116832,-0.075634,0.142574,N/A
2,AAP,229.87,N/A,0.533743,0.815842,0.207903,0.839604,0.122241,0.877228,0.007955,0.615842,N/A
3,AAPL,167.46,N/A,0.370301,0.653465,0.263445,0.889109,0.057965,0.722772,0.029574,0.734653,N/A
4,ABBV,122.53,N/A,0.169847,0.388119,0.055357,0.570297,-0.014060,0.437624,0.063361,0.859406,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.54,N/A,0.180956,0.411881,0.043751,0.542574,-0.055557,0.293069,-0.009572,0.508911,N/A
501,ZBH,124.10,N/A,-0.159926,0.043564,-0.252058,0.041584,-0.164400,0.073267,-0.138238,0.037624,N/A
502,ZBRA,608.50,N/A,0.582308,0.859406,0.189485,0.809901,0.004724,0.526733,0.111074,0.954455,N/A
503,ZION,67.12,N/A,0.698432,0.922772,0.128443,0.722772,0.104987,0.853465,0.027154,0.724752,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [28]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.04,N/A,0.354359,0.633663,0.105264,0.675248,-0.125512,0.126733,-0.024066,0.39802,0.458416
1,AAL,18.16,N/A,0.186820,0.427723,-0.272982,0.035644,-0.128236,0.116832,-0.075634,0.142574,0.180693
2,AAP,229.87,N/A,0.533743,0.815842,0.207903,0.839604,0.122241,0.877228,0.007955,0.615842,0.787129
3,AAPL,167.46,N/A,0.370301,0.653465,0.263445,0.889109,0.057965,0.722772,0.029574,0.734653,0.75
4,ABBV,122.53,N/A,0.169847,0.388119,0.055357,0.570297,-0.014060,0.437624,0.063361,0.859406,0.563861
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.54,N/A,0.180956,0.411881,0.043751,0.542574,-0.055557,0.293069,-0.009572,0.508911,0.439109
501,ZBH,124.10,N/A,-0.159926,0.043564,-0.252058,0.041584,-0.164400,0.073267,-0.138238,0.037624,0.04901
502,ZBRA,608.50,N/A,0.582308,0.859406,0.189485,0.809901,0.004724,0.526733,0.111074,0.954455,0.787624
503,ZION,67.12,N/A,0.698432,0.922772,0.128443,0.722772,0.104987,0.853465,0.027154,0.724752,0.805941


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [29]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,348.59,N/A,1.399781,0.980198,0.977110,0.990099,0.403946,0.974257,0.270719,0.986139,0.982673
1,AMD,165.34,N/A,0.788742,0.946535,0.974188,0.988119,0.405451,0.976238,0.280863,0.988119,0.974752
2,F,20.39,N/A,1.192289,0.978218,0.375532,0.958416,0.515530,0.988119,0.180449,0.974257,0.974752
3,ANET,131.10,N/A,0.849436,0.956436,0.482713,0.972277,0.352652,0.970297,0.232669,0.978218,0.969307
4,MCHP,86.40,N/A,1.460860,0.982178,1.087303,0.992079,1.057158,0.992079,0.081216,0.908911,0.968812
5,XLNX,238.70,N/A,0.663123,0.908911,0.800222,0.984158,0.440682,0.986139,0.252680,0.980198,0.964851
6,LB,80.25,N/A,2.357675,0.992079,0.836065,0.986139,0.224521,0.956436,0.079797,0.90297,0.959406
7,INTU,701.91,N/A,0.903993,0.966337,0.526613,0.976238,0.187621,0.932673,0.087166,0.924752,0.95
8,DVN,44.37,N/A,2.036408,0.990099,0.629784,0.982178,0.428595,0.982178,0.054554,0.831683,0.946535
9,ALB,286.38,N/A,0.980846,0.968317,0.629364,0.980198,0.132865,0.889109,0.086113,0.922772,0.940099


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [30]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [31]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price']) 
hqm_dataframe


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,348.59,57,1.399781,0.980198,0.977110,0.990099,0.403946,0.974257,0.270719,0.986139,0.982673
1,AMD,165.34,120,0.788742,0.946535,0.974188,0.988119,0.405451,0.976238,0.280863,0.988119,0.974752
2,F,20.39,980,1.192289,0.978218,0.375532,0.958416,0.515530,0.988119,0.180449,0.974257,0.974752
3,ANET,131.10,152,0.849436,0.956436,0.482713,0.972277,0.352652,0.970297,0.232669,0.978218,0.969307
4,MCHP,86.40,231,1.460860,0.982178,1.087303,0.992079,1.057158,0.992079,0.081216,0.908911,0.968812
5,XLNX,238.70,83,0.663123,0.908911,0.800222,0.984158,0.440682,0.986139,0.252680,0.980198,0.964851
6,LB,80.25,249,2.357675,0.992079,0.836065,0.986139,0.224521,0.956436,0.079797,0.90297,0.959406
7,INTU,701.91,28,0.903993,0.966337,0.526613,0.976238,0.187621,0.932673,0.087166,0.924752,0.95
8,DVN,44.37,450,2.036408,0.990099,0.629784,0.982178,0.428595,0.982178,0.054554,0.831683,0.946535
9,ALB,286.38,69,0.980846,0.968317,0.629364,0.980198,0.132865,0.889109,0.086113,0.922772,0.940099


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [32]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [33]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [34]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [35]:
writer.save()